# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.google.com'
}

In [3]:
url = f"https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,10"

In [4]:
response = requests.get(url, headers=headers)

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
soup.find_all('li', class_='ipc-metadata-list-summary-item')

[<li class="ipc-metadata-list-summary-item"><div class="ipc-metadata-list-summary-item__c"><div class="ipc-metadata-list-summary-item__tc"><span aria-disabled="false" class="ipc-metadata-list-summary-item__t"></span><div class="sc-74bf520e-3 avhjJ dli-parent"><div class="sc-74bf520e-2 WQsRg"><div class="sc-e5a25b0f-0 iVAGjF dli-poster-container"><div class="ipc-poster ipc-poster--base ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group"><div aria-label="add to watchlist" class="ipc-watchlist-ribbon ipc-focusable ipc-watchlist-ribbon--s ipc-watchlist-ribbon--base ipc-watchlist-ribbon--loading ipc-watchlist-ribbon--onImage ipc-poster__watchlist-ribbon" data-testid="poster-watchlist-ribbon-add" role="button" tabindex="0"><svg class="ipc-watchlist-ribbon__bg" height="34px" role="presentation" viewbox="0 0 24 34" width="24px" xmlns="http://www.w3.org/2000/svg"><polygon class="ipc-watchlist-ribbon__bg-ribbon" fill="#000000" points="24 0 0 0 0 32 12.2436611 26.2

In [19]:
def scrape_imdb(title_type, user_rating, start_date, end_date):
    # Construct the URL based on parameters
    url = f"https://www.imdb.com/search/title/?title_type={title_type}&release_date={start_date},{end_date}&user_rating={user_rating}&sort=num_votes,desc"

    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.google.com'
}
    
    # Send a request to the website
    response = requests.get(url , headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to load page: {url}")
    
    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Lists to store the scraped data
    movie_nrs, titles, years, ratings, runtimes, genres, descriptions, directors, stars, votes, gross = ([] for i in range(11))

    # Select the movie containers using 'li' elements with the new class 'ipc-metadata-list-summary-item'
    movie_containers = soup.find_all('li', class_='ipc-metadata-list-summary-item')

    for i, container in enumerate(movie_containers, 1):
        # Movie Number
        movie_nrs.append(i)
        
        # Title
        title = container.find('a', class_='ipc-metadata-list-summary-item__t').text if container.find('a', class_='ipc-metadata-list-summary-item__t') else 'NA'
        titles.append(title)

        # Year of Release
        year = container.find('span', class_='ipc-metadata-list-summary-item__st').text if container.find('span', class_='ipc-metadata-list-summary-item__st') else 'NA'
        years.append(year)

        # IMDb Rating
        rating = container.find('span', class_='ipc-metadata-list-summary-item__rating').text if container.find('span', class_='ipc-metadata-list-summary-item__rating') else 'NA'
        ratings.append(rating)

        # Runtime
        runtime = container.find('span', class_='ipc-metadata-list-summary-item__runtime').text.replace(' min', '') if container.find('span', class_='ipc-metadata-list-summary-item__runtime') else 'NA'
        runtimes.append(runtime)

        # Genre
        genre = container.find('span', class_='ipc-metadata-list-summary-item__genres').text.strip() if container.find('span', class_='ipc-metadata-list-summary-item__genres') else 'NA'
        genres.append(genre)

        # Description
        description = container.find('p', class_='ipc-metadata-list-summary-item__plot').text.strip() if container.find('p', class_='ipc-metadata-list-summary-item__plot') else 'NA'
        descriptions.append(description)

        # Director and Stars
        credits = container.find_all('li', class_='ipc-metadata-list-summary-item__crew')
        if len(credits) > 0:
            director = credits[0].text.replace('Director:', '').replace('Directors:', '').strip()
            directors.append(director)
        else:
            directors.append('NA')
        
        if len(credits) > 1:
            star = credits[1].text.replace('Stars:', '').strip()
            stars.append(star)
        else:
            stars.append('NA')

        # Votes and Gross
        nv = container.find_all('span', class_='ipc-metadata-list-summary-item__nv')
        vote = nv[0].text if len(nv) > 0 else 'NA'
        gross_earnings = nv[1].text.replace('$', '').replace('M', '') if len(nv) > 1 else 'NA'
        votes.append(vote)
        gross.append(gross_earnings)

    # Create DataFrame
    movies_df = pd.DataFrame({
        'Movie Nr': movie_nrs,
        'Title': titles,
        'Year': years,
        'Rating': ratings,
        'Runtime (min)': runtimes,
        'Genre': genres,
        'Description': descriptions,
        'Director': directors,
        'Stars': stars,
        'Votes': votes,
        'Gross ($M)': gross
    })

    return movies_df

In [20]:
df = scrape_imdb('feature', '7.5,10', '1990-01-01', '1992-12-31')
print(df.head())

   Movie Nr Title Year Rating Runtime (min) Genre Description Director Stars  \
0         1    NA   NA     NA            NA    NA          NA       NA    NA   
1         2    NA   NA     NA            NA    NA          NA       NA    NA   
2         3    NA   NA     NA            NA    NA          NA       NA    NA   
3         4    NA   NA     NA            NA    NA          NA       NA    NA   
4         5    NA   NA     NA            NA    NA          NA       NA    NA   

  Votes Gross ($M)  
0    NA         NA  
1    NA         NA  
2    NA         NA  
3    NA         NA  
4    NA         NA  


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here